In [1]:
# Run all cells to start, then come back to the top 
# When changing the selected nucleus or fit results, subsequent cells need to be rerun to update the output 

In [2]:
import os
import pickle
import numpy as np
pi=np.pi
from scipy.special import spherical_jn
#
import matplotlib.pyplot as plt
#
import pandas as pd
pd.set_option('display.max_rows', 500)
#
from IPython.display import display, Math
import ipywidgets as widgets
import re
#import time
#

# Methods

These are the methods to describe charge density, form factor, and radius

$ \rho (r) = \begin{cases} \sum^N_{n=1} a_n j_0(q_n r), &  r \leq R\\ 0,  & r > R  \end{cases} \quad \text{with} \quad q_n=\frac{\pi n}{R} $

In [3]:
def charge_density_FB(r,ai,R,N):
    nu=np.arange(1,N+1)
    qi = nu*pi/R
    #
    scalar=False
    if len(np.shape(r))==0:
        scalar=True
        r=np.array([r])
    #
    rho=np.array([0])
    #
    if np.any(r<=R):
        qr=np.einsum('i,j->ij',qi,r)
        rho = np.einsum('i,ij->j',ai,spherical_jn(0,qr))
    #
    if np.any(r>R):
        if np.size(rho)>1:
            rho[np.where(r>R)]=np.array([0])
        else:
            rho=np.array([0])
    #
    if scalar:
        rho=rho[0]
    #
    return rho

def charge_density_FB_uncertainty(r,ai,R,N,cov):
    nu=np.arange(1,N+1)
    qi = nu*pi/R
    #
    scalar=False
    if len(np.shape(r))==0:
        scalar=True
        r=np.array([r])
    #
    drho=np.array([0])
    #
    if np.any(r<=R):
        qr=np.einsum('i,j->ij',qi,r)
        drho_dai = spherical_jn(0,qr)
        drho=np.sqrt(np.einsum('ij,ik,kj->j',drho_dai,cov,drho_dai))
    #
    if np.any(r>R):
        if np.size(drho)>1:
            drho[np.where(r>R)]=np.array([0])
        else:
            drho=np.array([0])
    #
    if scalar:
        drho=drho[0]
    #
    return drho

$ F^\text{ch}(q) = \frac{4 \pi}{Z} R ~ j_0(q R) \sum^N_{n=1} (-1)^{n} \frac{a_n}{q^2 -q_n^2} \quad \text{with} \quad q_n=\frac{\pi n}{R} $

In [4]:
def formfactor_FB(q,ai,R,N,Z):
    nu=np.arange(1,N+1)
    qi = nu*pi/R
    #
    scalar=False
    if len(np.shape(q))==0:
        scalar=True
        q=np.array([q])
    #
    N_z=len(q)
    q_grid=np.tile(q,(N,1))
    qi_grid=np.tile(qi,(N_z,1)).transpose()
    denom=q_grid**2-qi_grid**2
    #
    F= 4*pi*R*spherical_jn(0,q*R)*np.einsum('i,ij->j',ai*(-1)**nu,1./denom)
    #
    if scalar:
        F=F[0]
    #
    return F/Z

def formfactor_FB_uncertainty(q,ai,R,N,Z,cov): 
    nu=np.arange(1,N+1)
    qi = nu*pi/R
    #
    scalar=False
    if len(np.shape(q))==0:
        scalar=True
        q=np.array([q])
    #
    N_z=len(q)
    q_grid=np.tile(q,(N,1))
    qi_grid=np.tile(qi,(N_z,1)).transpose()
    denom=q_grid**2-qi_grid**2
    #
    dF_dai = 4*pi*R*spherical_jn(0,q*R)*np.einsum('i,ij->ij',(-1)**nu,1./denom)
    dF=np.real(np.sqrt(np.einsum('ij,ik,kj->j',dF_dai,cov,dF_dai)+0j))
    #
    if scalar:
        dF=dF[0]
    #
    return dF/Z

$ \langle r^2 \rangle = \frac{4 \pi}{Z} \int_0^\infty \text{d}r' r'^4 \rho(r') = \frac{4 \pi}{Z} R^2 \sum^N_{n=1} (-1)^{n+1} \Big(\pi n - \frac{6}{\pi n}\Big) \frac{a_n}{q_n^3}$

$ r_\text{ch} = \sqrt{\langle r^2 \rangle} $

In [5]:
def charge_radius_FB(ai,Z,N):
    nu=np.arange(1,N+1)
    qi = nu*pi/R
    #
    Qi = (-1)**nu*nu*pi*(6-(nu*pi)**2)*ai/qi**5
    return np.sqrt(4*pi*np.sum(Qi)/Z) 

def charge_radius_FB_uncertainty(ai,Z,qi,N,R,cov):
    nu=np.arange(1,N+1)
    qi = nu*pi/R
    #
    r=charge_radius_FB(ai,Z,N)
    #
    nu=np.arange(1,N+1)
    dQi_dai = (-1)**nu*nu*pi*(6-(nu*pi)**2)/qi**5
    drsq_dai = 4*pi*dQi_dai/Z
    dr_dai=drsq_dai/(2*r)
    dr=np.sqrt(np.einsum('i,ik,k->',dr_dai,cov,dr_dai))
    return dr

# Loading data for the charge densities

We can load the file by simply unpickleing it:

In [6]:
def load_file(path,verbose=True):

    if os.path.exists(path):
        with open( path, "rb" ) as file:
            charge_dict = pickle.load(file) 
            if verbose:
                print("File loaded from "+path) 
    else:
        raise ValueError("No file found at "+path) 

    return charge_dict

In [7]:
Al27_pen_best = load_file('./Al27_best_woBarrett.pkl')
Al27_bar_best = load_file('./Al27_best_wBarrett.pkl')
#
Ca40_pen_best = load_file('./Ca40_best_woBarrett.pkl')
Ca40_bar_best = load_file('./Ca40_best_wBarrett.pkl')
#
Ca48_pen_best = load_file('./Ca48_best_woBarrett.pkl')
Ca48_bar_best = load_file('./Ca48_best_wBarrett.pkl')
#
Ti48_pen_best = load_file('./Ti48_best_woBarrett.pkl')
Ti48_bar_best = load_file('./Ti48_best_wBarrett.pkl')
#
Ti50_pen_best = load_file('./Ti50_best_woBarrett.pkl')
Ti50_bar_best = load_file('./Ti50_best_wBarrett.pkl')

File loaded from ./Al27_best_woBarrett.pkl
File loaded from ./Al27_best_wBarrett.pkl
File loaded from ./Ca40_best_woBarrett.pkl
File loaded from ./Ca40_best_wBarrett.pkl
File loaded from ./Ca48_best_woBarrett.pkl
File loaded from ./Ca48_best_wBarrett.pkl
File loaded from ./Ti48_best_woBarrett.pkl
File loaded from ./Ti48_best_wBarrett.pkl
File loaded from ./Ti50_best_woBarrett.pkl
File loaded from ./Ti50_best_wBarrett.pkl


In [8]:
nucleus_select = widgets.Dropdown(
    options=[('27Al', {'w/o':Al27_pen_best,'w/':Al27_bar_best}),('40Ca', {'w/o':Ca40_pen_best,'w/':Ca40_bar_best}),('48Ca', {'w/o':Ca48_pen_best,'w/':Ca48_bar_best}),('48Ti', {'w/o':Ti48_pen_best,'w/':Ti48_bar_best}),('50Ti', {'w/o':Ti50_pen_best,'w/':Ti50_bar_best})], 
    value= {'w/o':Al27_pen_best,'w/':Al27_bar_best},
    description='Nucleus:',
)

barrett_select = widgets.Dropdown(
    options=[('without Barrett moment', 'w/o'),('with Barrett moment', 'w/')],
    value= 'w/o',
    description='Barrett?:',
)

In [9]:
display(nucleus_select)
display(barrett_select)

Dropdown(description='Nucleus:', options=(('27Al', {'w/o': {'dataset': 'Li-AI', 'nucleus': 'Al27', 'method': '…

Dropdown(description='Barrett?:', options=(('without Barrett moment', 'w/o'), ('with Barrett moment', 'w/')), …

<ins> The loaded object is a dictionary with all relevant quantities listed. See for example: </ins>

In [10]:
best_fit = nucleus_select.value[barrett_select.value]
pd.DataFrame.from_dict(best_fit, orient="index")

0
dataset                                                                                  Li-AI
nucleus                                                                                   Al27
method                                                                                  powell
convergence_limit                                                                     0.000001
barrett_constraint                                                                       False
penalty_non_monotone                                                                      True
Z                                                                                           13
A                                                                                           27
m                                                                                 25133.143901
k                                                                                       2.0573
alpha                                                                                   0.0419
dof                                                                                         46
chisq                                                                                89.741842
redchisq                                                                               1.95091
p_val                                                                                 0.000121
resid                                        [1.226506516891039, 0.5347588489822335, -0.467...
R                                                                                          6.0
alim                                         [0.29700198429204083, 0.14850099214602042, 0.0...
N_x                                                                                          3
x                                            [0.6385541668674287, 0.5426914852739657, 0.389...
dx                                           [0.0012122652197065857, 0.000747209892826092, ...
cov_x                                        [[1.4695869629102563e-06, 5.931255437654337e-0...
N_a                                                                                          4
a                                            [0.061891226020341736, 0.053145760664733366, -...
da                                           [0.0001279327186950375, 0.000456018795407785, ...
cov_a                                        [[1.6366780512703596e-08, 5.763503323257725e-0...
r_ch                                                                                  2.995739
dr_ch                                                                                 0.007573
barrett                                                                                 8.3205
dbarrett                                                                              0.039641
dr_ch_syst                                                                            0.042974
dr_ch_syst_upper                                                                      0.033937
dr_ch_syst_lower                                                                      0.043145
dr_ch_model                                                                           0.044259
dbarrett_syst                                                                         0.224328
dbarrett_syst_upper                                                                   0.177354
dbarrett_syst_lower                                                                   0.225087
dbarrett_model                                                                         0.23107
dr_ch_dist                                                                            0.033481
dr_ch_dist_upper                                                                      0.026169
dr_ch_dist_lower                                                                      0.033481
dbarrett_dist                                                                         0.172703
dbarrett

We can directly extract the parameters and their uncertainties (note that 'da' is not scaled with $\sqrt{\text{red.}\chi^2}$ yet, but the statistical components inside 'da_model' are). See for example

In [11]:
display(nucleus_select)
display(barrett_select)

Dropdown(description='Nucleus:', options=(('27Al', {'w/o': {'dataset': 'Li-AI', 'nucleus': 'Al27', 'method': '…

Dropdown(description='Barrett?:', options=(('without Barrett moment', 'w/o'), ('with Barrett moment', 'w/')), …

In [12]:
best_fit = nucleus_select.value[barrett_select.value]
pd.DataFrame(np.stack([best_fit['a'],best_fit['da']*np.sqrt(best_fit['redchisq']),best_fit['da_syst'],best_fit['da_model']],axis=-1),columns=['a','da_stat','da_syst','da_comb'],index=[r"$a_{{ {i} }}$".format(i=i) for i in range(1,best_fit['N_a']+1)])

a   da_stat   da_syst   da_comb
$a_{ 1 }$  0.061891  0.000179  0.000718  0.000740
$a_{ 2 }$  0.053146  0.000637  0.002385  0.002465
$a_{ 3 }$ -0.021248  0.000296  0.003263  0.003274
$a_{ 4 }$ -0.016408  0.000259  0.005495  0.005501

# Plotting charge density and form factor

In [13]:
def plot_charge_density(r,charge_dict,label=None):
    ai=charge_dict['a']
    R=charge_dict['R']
    N=charge_dict['N_a']
    #
    rho=charge_density_FB(r,ai,R,N)
    #
    cov_a_u = charge_dict['cov_a_model_upper']
    cov_a_l = charge_dict['cov_a_model_lower']
    #
    drho_u=charge_density_FB_uncertainty(r,ai,R,N,cov_a_u)
    drho_l=charge_density_FB_uncertainty(r,ai,R,N,cov_a_l)
    #
    plt.plot(r,rho,label=label)
    plt.fill_between(r, rho+drho_u, rho-drho_l,alpha=0.5)
    #

In [14]:
display(nucleus_select)

Dropdown(description='Nucleus:', options=(('27Al', {'w/o': {'dataset': 'Li-AI', 'nucleus': 'Al27', 'method': '…

In [15]:
best_fit_wo = nucleus_select.value['w/o']
best_fit_w = nucleus_select.value['w/']

name = '$^{'+str(best_fit_wo['A'])+'}$'+ re.findall('\D+', nucleus_select.label)[0]

plt.title(r'charge density of '+name)
r=np.arange(0,8,0.1)
plot_charge_density(r,best_fit_wo,label='w/o Barrett')
plot_charge_density(r,best_fit_w,label='w/  Barrett')
plt.legend()
plt.xlim(0,8)
plt.xlabel(r'$r$ in fm')
plt.ylim(0,0.09)
plt.ylabel(r'$\rho(r)$ in fm$^{-3}$')
plt.show()

In [16]:
def plot_formfactor(q,charge_dict,label=None):
    ai=charge_dict['a']
    R=charge_dict['R']
    N=charge_dict['N_a']
    Z=charge_dict['Z']
    #
    FF=formfactor_FB(q,ai,R,N,Z)
    #
    cov_a = charge_dict['cov_a_model']
    #
    dFF=formfactor_FB_uncertainty(q,ai,R,N,Z,cov_a)
    #
    plt.plot(q,np.abs(FF),label=label)
    plt.fill_between(q, np.abs(FF)-dFF, np.abs(FF)+dFF,alpha=0.5)
    #

In [17]:
display(nucleus_select)

Dropdown(description='Nucleus:', options=(('27Al', {'w/o': {'dataset': 'Li-AI', 'nucleus': 'Al27', 'method': '…

In [28]:
best_fit_wo = nucleus_select.value['w/o']
best_fit_w = nucleus_select.value['w/']

name = '$^{'+str(best_fit_wo['A'])+'}$'+ re.findall('\D+', nucleus_select.label)[0]

plt.title(r'charge form factor of '+name)
q=np.arange(0,3,1e-2)
plot_formfactor(q,best_fit_wo,label='w/o Barrett')
plot_formfactor(q,best_fit_w,label='w/  Barrett')
plt.legend()
plt.xlim(0,3)
plt.xlabel(r'$q$ in fm$^{-1}$')
plt.yscale('log')
plt.ylim(5e-5,1)
plt.ylabel(r'$\rho(r)$ in fm$^{-3}$')
plt.show()

The values for the radii and all uncertainties can also be taken directly from the dictionary:

In [19]:
def print_radius(charge_dict):
    redchisq=charge_dict['redchisq']
    
    r_ch=charge_dict['r_ch']
    
    dr=charge_dict['dr_ch']*np.sqrt(redchisq)
    dr_s=charge_dict['dr_ch_syst']
    dr_m=charge_dict['dr_ch_model']
    dr_u=charge_dict['dr_ch_syst_upper']
    dr_l=charge_dict['dr_ch_syst_lower']
    dr_d=charge_dict['dr_ch_dist']
    dr_d_u=charge_dict['dr_ch_dist_upper']
    dr_d_l=charge_dict['dr_ch_dist_lower']
    dr_dm=np.sqrt(dr**2+dr_d**2)
    
    digs_r=int(-np.log10(dr))+2
    
    while np.max([dr,dr_s,dr_m])*10**digs_r>69: 
        digs_r-=1
    while np.max([dr,dr_s,dr_m])*10**digs_r<3.1:
        digs_r+=1
        
    s=r'\sqrt{\langle r^2 \rangle} = '+r'{r_ch:1.{digs_r}f}({dr:1d})'.format(r_ch=r_ch,digs_r=digs_r,dr=int(np.round(dr*10**digs_r,0)))+r'\begin{matrix}'+r'({dr_s:1d}) [{dr_m:1d}] \\ ( +{dr_d_u:1d}/-{dr_d_l:1d} ) [{dr_dm:1d}]'.format(dr_s=int(np.round(dr_s*10**digs_r,0)),dr_m=int(np.round(dr_m*10**digs_r,0)),dr_d_u=int(np.round(dr_d_u*10**digs_r,0)),dr_d_l=int(np.round(dr_d_l*10**digs_r,0)),dr_dm=int(np.round(dr_dm*10**digs_r,0)))+r'\end{matrix}'
    display(Math(s))


In [20]:
display(nucleus_select)
display(barrett_select)

Dropdown(description='Nucleus:', options=(('27Al', {'w/o': {'dataset': 'Li-AI', 'nucleus': 'Al27', 'method': '…

Dropdown(description='Barrett?:', options=(('without Barrett moment', 'w/o'), ('with Barrett moment', 'w/')), …

In [21]:
best_fit = nucleus_select.value[barrett_select.value]
print_radius(best_fit)

<IPython.core.display.Math object>

# Calculating and propagaing uncertainties

To show how to propagate / calculate uncertainties yourself we reconstruct the charge radius uncertainties in the following section. For the upper lower limit based on all sufficient solutions, we need to load the dictionaries with all remaining solutions:

In [22]:
Al27_pen_syst = load_file('./Al27_syst_woBarrett.pkl')
Al27_bar_syst = load_file('./Al27_syst_wBarrett.pkl')
#
Ca40_pen_syst = load_file('./Ca40_syst_woBarrett.pkl') 
Ca40_bar_syst = load_file('./Ca40_syst_wBarrett.pkl') 
#
Ca48_pen_syst = load_file('./Ca48_syst_woBarrett.pkl') 
Ca48_bar_syst = load_file('./Ca48_syst_wBarrett.pkl') 
#
Ti48_pen_syst = load_file('./Ti48_syst_woBarrett.pkl')
Ti48_bar_syst = load_file('./Ti48_syst_wBarrett.pkl')
#
Ti50_pen_syst = load_file('./Ti50_syst_woBarrett.pkl')
Ti50_bar_syst = load_file('./Ti50_syst_wBarrett.pkl')

File loaded from ./Al27_syst_woBarrett.pkl
File loaded from ./Al27_syst_wBarrett.pkl
File loaded from ./Ca40_syst_woBarrett.pkl
File loaded from ./Ca40_syst_wBarrett.pkl
File loaded from ./Ca48_syst_woBarrett.pkl
File loaded from ./Ca48_syst_wBarrett.pkl
File loaded from ./Ti48_syst_woBarrett.pkl
File loaded from ./Ti48_syst_wBarrett.pkl
File loaded from ./Ti50_syst_woBarrett.pkl
File loaded from ./Ti50_syst_wBarrett.pkl


In [29]:
nucleus_select_2 = widgets.Dropdown(
    options=[('27Al', {'w/o':[Al27_pen_best,Al27_pen_syst],'w/':[Al27_bar_best,Al27_bar_syst]}), ('40Ca', {'w/o':[Ca40_pen_best,Ca40_pen_syst],'w/':[Ca40_bar_best,Ca40_bar_syst]}), ('48Ca', {'w/o':[Ca48_pen_best,Ca48_pen_syst],'w/':[Ca48_bar_best,Ca48_bar_syst]}), ('48Ti', {'w/o':[Ti48_pen_best,Ti48_pen_syst],'w/':[Ti48_bar_best,Ti48_bar_syst]}), ('50Ti', {'w/o':[Ti50_pen_best,Ti50_pen_syst],'w/':[Ti50_bar_best,Ti50_bar_syst]})], 
    value= {'w/o':[Al27_pen_best,Al27_pen_syst],'w/':[Al27_bar_best,Al27_bar_syst]},
    description='Nucleus:',
)

barrett_select_2 = widgets.Dropdown(
    options=[('without Barrett moment', 'w/o'),('with Barrett moment', 'w/')],
    value= 'w/o',
    description='Barrett?:',
)

To start clean from the uncertainties and properties of $x_i$ we need the formulas to transform x to a:

$ a_n(x_n) = -(-1)^n \Big[ x_n (\tilde{a}_{n, \overline{\text{max}}} - \tilde{a}_{n, \overline{\text{min}}}) + \tilde{a}_{n, \overline{\text{min}}} \Big] $

$ \tilde{a}_{n, \overline{\text{max}}} = \min\left(\tilde{a}_{n, \text{lim}},\frac{Z}{4 \pi R} q_n^2 + \sum^{N}_{\substack{n' > n \\ n'=1}} \tilde{a}_{n',\text{lim}} \frac{q_n^2}{q_{n'}^2} - \sum^{N}_{\substack{n' < n \\ n'=1}} \tilde{a}_{n'} \frac{q_n^2}{q_{n'}^2} \right) $  

$ \tilde{a}_{n, \overline{\text{min}}} = \max\left(-\tilde{a}_{n, \text{lim}},\frac{Z}{4 \pi R} q_n^2 - \sum^{N}_{\substack{n' > n \\ n'=1}} \tilde{a}_{n',\text{lim}} \frac{q_n^2}{q_{n'}^2} - \sum^{N}_{\substack{n' < n \\ n'=1}} \tilde{a}_{n'} \frac{q_n^2}{q_{n'}^2} \right) $

$ \tilde{a}_{n,\text{lim}} = \frac{1}{n} \frac{\pi^2 Z}{2 R^3} $

(alternatively one can start directly with the values for the $a_i$, however, one has a different correlation matrix for every uncertainty)

In [24]:
def ai_xi(xi,ai_tilde_lim,Z,R,Cov=None):

    N=len(ai_tilde_lim)
    
    ai_t=np.zeros(N)
    
    if Cov is not None:
        J_t=np.zeros((N,N-1))

    i_a0=0

    for i in range(1,N+1):
        ai_tilde_min, ai_tilde_max = ai_impl_border(i,ai_t[i_a0:],ai_tilde_lim,Z,R)
        i_a = i - 1
        ai_t[i_a]=xi[i-1]*(ai_tilde_max-ai_tilde_min) + ai_tilde_min if i<N else ai_tilde_min
        if Cov is not None:
            for j in range(1,N):
                i_range=np.arange(1,i)
                J_t[i_a,i_a0+j-1]= ( ((ai_tilde_max-ai_tilde_min) if i==j else 0) + ( -xi[i-1]*np.sum(J_t[i_a0:i_a,i_a0+j-1]*(i/i_range)**2)  if ai_tilde_max<ai_tilde_lim[i-1] else 0 ) + ( - (1-xi[i-1])*np.sum(J_t[i_a0:i_a,i_a0+j-1]*(i/i_range)**2)  if -ai_tilde_lim[i-1]<ai_tilde_min else 0 ) ) if i<N else - np.sum(J_t[i_a0:i_a,i_a0+j-1]*(i/i_range)**2)

    ai=np.copy(ai_t)
    ai[i_a0:]=ai_tilde(ai_t[i_a0:])

    if Cov is not None:
        nus=np.arange(1,N+1)
        J = np.einsum('ij,i->ij',J_t,(-1)**(nus+1))
        Cov_ai = J @ Cov @ np.transpose(J) #np.einsum('ij,jk,lk->il',J,C,J)
        return ai, Cov_ai#, J
    else:
        return ai

def ai_impl_border(j,ai,ai_lim,Z,R):

    N=len(ai_lim)
    nus=np.arange(1,N+1)
    qnus=nus*pi/R
    qj=qnus[j-1]

    mean = Z/(4*pi*R) - np.sum(ai[0:j-1]/qnus[0:j-1]**2)
    spread = np.sum(ai_lim[j:N+1]/qnus[j:N+1]**2)

    impl_min=min(qj**2*(mean + spread),ai_lim[j-1])
    impl_max=max(qj**2*(mean - spread),-ai_lim[j-1])

    return  impl_max , impl_min

def ai_tilde(ai):
    N=len(ai)
    nus=np.arange(1,N+1)
    return ai*(-1)**(nus+1)

In [25]:
display(nucleus_select_2)
display(barrett_select_2)

Dropdown(description='Nucleus:', options=(('27Al', {'w/o': [{'dataset': 'Li-AI', 'nucleus': 'Al27', 'method': …

Dropdown(description='Barrett?:', options=(('without Barrett moment', 'w/o'), ('with Barrett moment', 'w/')), …

In [30]:
best_fit=nucleus_select_2.value[barrett_select_2.value][0]
fits_syst=nucleus_select_2.value[barrett_select_2.value][1]
#
xi=best_fit['x']
alim=best_fit['alim']
R = best_fit['R']
Z = best_fit['Z']
ai = best_fit['a']
N = best_fit['N_x']
nu=np.arange(1,N+1+1)
qi = nu*pi/R
k, alpha = best_fit['k'], best_fit['alpha']
#
# The uncertainties from the fit or the paramterization of the systematic uncertainties can be propagated normally 
#
dxi=best_fit['dx']
dxi_syst=best_fit['dx_syst']
dxi_syst_u=best_fit['dx_syst_upper']
dxi_syst_l=best_fit['dx_syst_lower']
#
redchisq = best_fit['redchisq']
dxi_model = np.sqrt(redchisq*dxi**2+dxi_syst**2) #the statistical part inside dxi_model is already rescaled with sqrt(red.chi^2)
dxi_model_u = np.sqrt(redchisq*dxi**2+dxi_syst_u**2)
dxi_model_l = np.sqrt(redchisq*dxi**2+dxi_syst_l**2)
#
cov_xi=best_fit['cov_x']
corr_xi=np.einsum('i,ij,j->ij',1./dxi,cov_xi,1./dxi)
#
cov_xi_syst=np.einsum('i,ij,j->ij',dxi_syst,corr_xi,dxi_syst)
cov_xi_syst_u=np.einsum('i,ij,j->ij',dxi_syst_u,corr_xi,dxi_syst_u)
cov_xi_syst_l=np.einsum('i,ij,j->ij',dxi_syst_l,corr_xi,dxi_syst_l)
cov_xi_model=np.einsum('i,ij,j->ij',dxi_model,corr_xi,dxi_model)
cov_xi_model_u=np.einsum('i,ij,j->ij',dxi_model_u,corr_xi,dxi_model_u)
cov_xi_model_l=np.einsum('i,ij,j->ij',dxi_model_l,corr_xi,dxi_model_l)
#
_,cov_ai_syst=ai_xi(xi,alim,Z,R,Cov=cov_xi_syst)
_,cov_ai_syst_u=ai_xi(xi,alim,Z,R,Cov=cov_xi_syst_u)
_,cov_ai_syst_l=ai_xi(xi,alim,Z,R,Cov=cov_xi_syst_l)
_,cov_ai_model=ai_xi(xi,alim,Z,R,Cov=cov_xi_model)
_,cov_ai_model_u=ai_xi(xi,alim,Z,R,Cov=cov_xi_model_u)
_,cov_ai_model_l=ai_xi(xi,alim,Z,R,Cov=cov_xi_model_l)
#
dai_syst=np.sqrt(np.diagonal(cov_ai_syst))
dai_syst_u=np.sqrt(np.diagonal(cov_ai_syst_u))
dai_syst_l=np.sqrt(np.diagonal(cov_ai_syst_l))
dai_model=np.sqrt(np.diagonal(cov_ai_model))
dai_model_u=np.sqrt(np.diagonal(cov_ai_model_u))
dai_model_l=np.sqrt(np.diagonal(cov_ai_model_l))
#
dr_syst=charge_radius_FB_uncertainty(ai,Z,qi,N+1,R,cov_ai_syst)
dr_syst_u=charge_radius_FB_uncertainty(ai,Z,qi,N+1,R,cov_ai_syst_u)
dr_syst_l=charge_radius_FB_uncertainty(ai,Z,qi,N+1,R,cov_ai_syst_l)
dr_model=charge_radius_FB_uncertainty(ai,Z,qi,N+1,R,cov_ai_model)
#
# We can also calculate upper and lower limits based on all remaining solutions
#
r=best_fit['r_ch']
#
rmax=r
rmin=r
#
for key in fits_syst:
    ri = fits_syst[key]['r_ch']
    if ri>rmax:
        rmax=ri
    elif ri<rmin:
        rmin=ri
#
dr_dist_u = rmax-r
dr_dist_l = r-rmin
dr_dist = np.max([dr_dist_u,dr_dist_l],axis=0)
#
dr=best_fit['dr_ch']*np.sqrt(redchisq)
dr_dist_model=np.sqrt(dr**2+dr_dist**2)
#

In [27]:
r_ch=r
dr=dr
dr_s=dr_syst
dr_m=dr_model
dr_u=dr_syst_u
dr_l=dr_syst_l
dr_d=dr_dist
dr_d_u=dr_dist_u
dr_d_l=dr_dist_l
dr_dm=dr_dist_model

digs_r=int(-np.log10(dr))+2

while np.max([dr,dr_s,dr_m])*10**digs_r>69: 
    digs_r-=1
while np.max([dr,dr_s,dr_m])*10**digs_r<3.1:
    digs_r+=1
    
s=r'\sqrt{\langle r^2 \rangle} = '+r'{r_ch:1.{digs_r}f}({dr:1d})'.format(r_ch=r_ch,digs_r=digs_r,dr=int(np.round(dr*10**digs_r,0)))+r'\begin{matrix}'+r'({dr_s:1d}) [{dr_m:1d}] \\ ( +{dr_d_u:1d}/-{dr_d_l:1d} ) [{dr_dm:1d}]'.format(dr_s=int(np.round(dr_s*10**digs_r,0)),dr_m=int(np.round(dr_m*10**digs_r,0)),dr_d_u=int(np.round(dr_d_u*10**digs_r,0)),dr_d_l=int(np.round(dr_d_l*10**digs_r,0)),dr_dm=int(np.round(dr_dm*10**digs_r,0)))+r'\end{matrix}'
display(Math(s))


<IPython.core.display.Math object>

As can be seen this results in the same values as directly extracted from the dictionary. 